### Пример: логистическая регрессия на Pytorch

$\hat{y} = \sigma(w^T x + b)$

$\sigma(t) = \frac{1}{1 + \exp(-t)}$

$\text{CE}(y, \hat{y}) = -y \cdot \log \hat{y} - (1 - y) \log (1 - \hat{y})$

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" style="background:white" width="600"/>

In [1]:
import torch

In [2]:
torch.manual_seed(42)
n_samples = 16
n_features = 5
x = torch.randn(n_samples, n_features)  # входной тензор
y = torch.randint(2, size=(n_samples, 1)).float()  # выходной тензор
w = torch.randn(
    n_features, 1, requires_grad=True
)  # параметр, хотим обновлять градиентным спуском
b = torch.randn(1, requires_grad=True)  # параметр, хотим обновлять градиентным спуском
z = torch.matmul(x, w) + b
y_hat = torch.sigmoid(z)
loss = (-y * y_hat.log() - (1 - y) * (1 - y_hat).log()).mean()
# loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)  # функция потерь, хотим минимизировать
loss

tensor(0.8950, grad_fn=<MeanBackward0>)

In [3]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x10dd07e80>
Gradient function for loss = <MeanBackward0 object at 0x10dd07c10>


Напишем цикл для поиска параметров, минимизирующих функцию ошибки

In [4]:
n_iter = 1000
step = 0.1
for i in range(n_iter):
    z = x @ w + b
    loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
    # посчитаем производные
    loss.backward()

    # обновим значения параметров внутри менеджера контекста `no_grad`
    with torch.no_grad():
        w -= w.grad * step
        b -= b.grad * step

    # обнулим градиенты, мы не хотим их накапливать в данном случае
    w.grad = None
    b.grad = None

# посмотрим, уменьшилось ли значение ошибки
print(loss)

tensor(0.3861, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
